Что нужно допилить:
1. Необходимо везде проверить старший и младший байты
2. Дописать TW и THx09 (тот, который писал Игорь, это THx05)
3. У TD и TW проверить правильно ли считается сглаживание
4. Блютуз также видит датчики TG, но в той методичке их нет, возможно их тоже нужно дописать
5. Поставить диапазон устройств
6. Поставить счетчик устройств (их кол-во)
7. Добавить сортировку по RSSI
8. Выгрузить отсканированные датчики в CSV

In [22]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
from bleak import BleakScanner

print('Выберите тип датчика из списка:   ')
detector_type = str(input('TD, TT, TL, DU, TH, TW, TP'))
range_start = int(input('Введите нижнюю границу: '))
range_end = int(input('Введите верхнюю границу: '))
searching_time = float(input('Введите время поиска LBE-устройств: '))
print()

# Функции распарсинга данный для датчиков
def TH(device):
    data = device.metadata['manufacturer_data'][3862]
    print("Device %s (%s), RSSI=%d dB" % (device.address, device.name, device.rssi))

    # Расшифровка температуры
    temp_raw = int.from_bytes(data[1:3], byteorder='little', signed=True)
    temp = temp_raw / 10.0
    print(f"Температура: {temp} градусов Цельсия")

    # Расшифровка освещенности
    light_raw = int.from_bytes(data[3:5], byteorder='little', signed=False)
    print(f"Освещенность: {light_raw} люкс")

    # Расшифровка влажности
    humidity_raw = int.from_bytes(data[5:7], byteorder='little', signed=False)
    humidity = humidity_raw / 10.0
    print(f"Влажность: {humidity}%")

    # Расшифровка атмосферного давления
    pressure_raw = int.from_bytes(data[7:9], byteorder='little', signed=False)
    pressure = pressure_raw + 55000
    print(f"Атмосферное давление: {pressure} Па")

    # Расшифровка режима работы
    mode_raw = int.from_bytes(data[9:10], byteorder='little', signed=False)
    period = (mode_raw & 0b00000111) * 20
    print(f"Период опроса датчиков: {period} секунд")
    flags = mode_raw & 0b11111000
    if flags & 0b10000000:
        print("Датчик температуры включен")
    if flags & 0b01000000:
        print("Датчик влажности включен")
    if flags & 0b00100000:
        print("Датчик атмосферного давления включен")
    if flags & 0b00010000:
        print("Датчик освещенности включен")
    if flags & 0b00001000:
        print("Датчик холла сработал")

        # Расшифровка напряжения батареи
        battery_raw = int.from_bytes(data[10:11], byteorder='little', signed=False)
        battery = battery_raw / 10.0
        print(f"Напряжение батареи: {battery} В")

        # Расшифровка версии прошивки
        version_raw = int.from_bytes(data[11:12], byteorder='little', signed=False)
        print(f"Версия прошивки: {version_raw}")
        print()

def TD(device):
    data = device.metadata['manufacturer_data'][3862]
    print("Device %s (%s), RSSI=%d dB" % (device.address, device.name, device.rssi))
    
    # Расшифровка уровня топлива
    oil_level_raw = int.from_bytes(data[1:3], byteorder='little', signed=False)
    print(f"Уровень топлива: {oil_level_raw}")

    # Расшифровка напряжения встроенной батареи
    battery_voltage_raw = int.from_bytes(data[3:4], byteorder='little', signed=False)
    battery_voltage = battery_voltage_raw / 10.0
    print(f"Напряжение встроенной батареи: {battery_voltage} В")

    # Расшифровка температуры
    TD_temp_raw = int.from_bytes(data[4:5], byteorder='little', signed=True)
    print(f"Температура: {TD_temp_raw} Градус Цельсия")

    # Расшифровка версии прошивки
    TD_version_raw = int.from_bytes(data[5:6], byteorder='little', signed=False)
    print(f"Версия прошивки: {TD_version_raw}")
                    
    # Расшифровка периода (непосредственно измеренный параметр)
    TD_period_param = int.from_bytes(data[6:10], byteorder='little', signed=False)
    print(f"Период: {TD_period_param} ")

    # Расшифровка режима работы
    TD_mode_raw = int.from_bytes(data[10:11], byteorder='little', signed=False)
    TD_range = (TD_mode_raw & 0b11110000)
    if TD_range & 0x80:
        print("Диапазон датчика 4095")
    if TD_range & 0b00000000:
        print("Диапазон датчика 1024")

    TD_flags = (TD_mode_raw & 0b00001111)
    if TD_flags & 0b00000000:
        print("Обычный алгоритм расчета")
    if TD_flags & 0b00000001:
        print("Алгоритм расчета с тарировочной таблицей")

    # Расшифровка сглаживания и термокомпенсации
    TD_smoth_and_copm_raw = int.from_bytes(data[11:12], byteorder='little', signed=False)
    TD_therm_compensation = (TD_smoth_and_copm_raw & 0b11110000)
    if TD_therm_compensation & 0x80:
        print("Термокомпенсация выключена")
    if TD_therm_compensation & 0b00000000:
        print("Термокомпенсация включена")

    TD_smoothing = TD_smoth_and_copm_raw & 0x0F
    print(f'Сглаживание: {TD_smoothing}')
    print('нужно проверить как работает расчет сглаживания')
    print()   
    
def DU(device):
    data = device.metadata['manufacturer_data'][3862]
    print("Device %s (%s), RSSI=%d dB" % (device.address, device.name, device.rssi))

    # Расшифровка режима датчика угла
    angle_mode_raw = int.from_bytes(data[1:2], byteorder='big', signed=False)
    print()
    print(f"Режим угла: {angle_mode_raw}")

    # Расшифровка события сработки
    event_raw = int.from_bytes(data[2:3], byteorder='big', signed=False)
    event = event_raw
    #потом надо прикрутить, что если угол больше 2 то приходит уведомление
    if event > 2 or event < 0:
        print(f'Событие: угол превышен {event}')
    else:
        print(f"Событие: {event}")
                    
    # Расшифровка угла наклона/количества оборотов
    DU_tilt_angle = int.from_bytes(data[3:5], byteorder='little', signed=False)
    print(f"Угол наклона/кол-во оборотов: {DU_tilt_angle} градусов")

    # Расшифровка температуры
    DU_version_raw = int.from_bytes(data[11:12], byteorder='big', signed=False)
    DU_temp_raw = int.from_bytes(data[5:6], byteorder='big', signed=True)
    if TD_version_raw == 102:
        print('Температура не используется в 102 версии прошивки')  
    else:  
        print(f"Температура: {DU_temp_raw} Градус Цельсия")
                    
    # Расшифровка верхней калибровки угла
    up_angle_calibration = int.from_bytes(data[6:8], byteorder='big', signed=False)
    print(f"Нижняя калибровка угла: {up_angle_calibration} градусов")
                    
    # Расшифровка нижней калибровки угла
    down_angle_calibration = int.from_bytes(data[8:10], byteorder='big', signed=False)
    print(f"Нижняя калибровка угла: {down_angle_calibration} градусов")
                    
    # Расшифровка напряжения встроенной батареи
    DU_battery_raw = int.from_bytes(data[10:11], byteorder='big', signed=False)
    DU_battery = DU_battery_raw / 10.0
    print(f"Напряжение батареи: {DU_battery} В")

    # Расшифровка версии прошивки
    DU_version_raw = int.from_bytes(data[11:12], byteorder='big', signed=False)
    print(f"Номер версии прошивки: {TD_version_raw}")
    
def TT(device):    
    data = device.metadata['manufacturer_data'][3862]
    print("Device %s (%s), RSSI=%d dB" % (device.address, device.name, device.rssi))

    # Расшифровка температуры
    TT_temp_raw = int.from_bytes(data[1:3], byteorder='little', signed=True)
    TT_temp = TT_temp_raw / 10.0
    print(f"Температура: {TT_temp} градусов Цельсия")
                    
    # Расшифровка напряжения встроенной батареи
    TT_battery_raw = int.from_bytes(data[3:4], byteorder='big', signed=False)
    TT_battery = TT_battery_raw / 10.0
    print(f"Напряжение батареи: {TT_battery} В")

    # Расшифровка версии прошивки
    TT_version_raw = int.from_bytes(data[4:5], byteorder='big', signed=False)
    print(f"Номер версии прошивки: {TT_version_raw}")
  
def TL(device):
    data = device.metadata['manufacturer_data'][3862]
    print("Device %s (%s), RSSI=%d dB" % (device.address, device.name, device.rssi))

    # Расшифровка температуры
    TL_temp_raw = int.from_bytes(data[1:3], byteorder='little', signed=True)
    TL_temp = TL_temp_raw / 10.0
    print(f"Температура: {TL_temp} градусов Цельсия")
                    
    # Расшифровка освещенности
    TL_light_raw = int.from_bytes(data[3:5], byteorder='little', signed=False)
    print(f"Освещенность: {TL_light_raw} люкс")
                    
    # Расшифровка напряжения встроенной батареи
    TL_battery_raw = int.from_bytes(data[3:4], byteorder='big', signed=False)
    TL_battery = TL_battery_raw / 10.0
    print(f"Напряжение батареи: {TL_battery} В")

    # Расшифровка версии прошивки
    TT_version_raw = int.from_bytes(data[4:5], byteorder='big', signed=False)
    print(f"Номер версии прошивки: {TT_version_raw}")
    
async def run():
    devices = await BleakScanner.discover(timeout=searching_time)
    for device in devices:
        if 'manufacturer_data' in device.metadata and device.name is not None:
            if device.name.startswith(detector_type):
                device_number = int(device.name[4:10])
                if range_start <= device_number <= range_end:            
                    if detector_type == 'TH':
                        TH(device)
                    elif detector_type == 'TD':
                        TD(device)
                    elif detector_type == 'DU':
                        DU(device)
                    elif detector_type == 'TT':
                        TT(device)
                    elif detector_type == 'TL':
                        TL(device)
                    else:
                        print('это устройство мы еще не внесли')
                else:
                    print('есть устройства за диапазоном указанных значений')
                    
                

loop = asyncio.get_event_loop()
loop.run_until_complete(run())

Выберите тип датчика из списка:   
TD, TT, TL, DU, TH, TW, TPTD
Введите нижнюю границу: 0
Введите верхнюю границу: 100
Введите время поиска LBE-устройств: 5

есть устройства за диапазоном указанных значений
есть устройства за диапазоном указанных значений
есть устройства за диапазоном указанных значений
есть устройства за диапазоном указанных значений
есть устройства за диапазоном указанных значений
есть устройства за диапазоном указанных значений
есть устройства за диапазоном указанных значений
есть устройства за диапазоном указанных значений
есть устройства за диапазоном указанных значений
есть устройства за диапазоном указанных значений
есть устройства за диапазоном указанных значений
есть устройства за диапазоном указанных значений
Device F1:16:A5:0C:24:64 (TD_100001), RSSI=-84 dB
Уровень топлива: 0
Напряжение встроенной батареи: 3.7 В
Температура: 22 Градус Цельсия
Версия прошивки: 5
Период: 3936000 
Сглаживание: 0
нужно проверить как работает расчет сглаживания

есть устройства з

C:\Users\timya\AppData\Local\Temp\ipykernel_7864\2856803349.py:206: FutureWarning: BLEDevice.metadata is deprecated and will be removed in a future version of Bleak, use AdvertisementData instead
  if 'manufacturer_data' in device.metadata and device.name is not None:
C:\Users\timya\AppData\Local\Temp\ipykernel_7864\2856803349.py:65: FutureWarning: BLEDevice.metadata is deprecated and will be removed in a future version of Bleak, use AdvertisementData instead
  data = device.metadata['manufacturer_data'][3862]
C:\Users\timya\AppData\Local\Temp\ipykernel_7864\2856803349.py:66: FutureWarning: BLEDevice.rssi is deprecated and will be removed in a future version of Bleak, use AdvertisementData.rssi instead
  print("Device %s (%s), RSSI=%d dB" % (device.address, device.name, device.rssi))
